In [3]:
# Click into this cell and press shift-enter before using this notebook.
# This line loads the ability to use %%ai in your file
%load_ext jupyter_ai_magics
# These lines import the Python modules we commonly use in CMPSC 5A
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots

# Lecture 10, CMPSC 5A, S25

Thu 05/01/2025, Week 5

## Updates on ic's

### ic12

* ic12 is almost fully graded as of noon (110/128).   I'll be finished grading it soon.
  * I gave credit for ic12 if you got all five activities done in *any* of the files in either the ic10 folders from Tuesday 04/22 or the ic12 folders from Thursday 04/24
  * If you did get all five activities done before 2pm today (Thu May 1st), but you got a grade less than 100, please submit a private EdPost (not an email) and ask for a regrade by midnight tonight, 05/02/2025.  Include a link to the file where all of this was completed.

### ic13

* ic13: This was the participation activity from Tuesday.  It's fully graded.
  * The instructions for this appear in the lecture09 notebook which you can access by updating your
    JupyterHub (via the link on canvas) and looking in the lectures folder.
  * I gave credit if I found a post from you int the [Week 4 Chat Group Folder](https://docs.google.com/spreadsheets/d/1X7GF5QxbYTAauvdKb-7LFD3zUwj2oqFMBi6Q8RHNjoM/edit?usp=sharing) that followed the instructions.
  * I was pretty generous; you were supposed to include a link to the folder where you did the ic10/ic12 work if you said that you had completed it.  Not everyone did, but I gave credit anyway.
  * If you got a zero on this, you can make it up if you (a) did the required posts before 2pm May 1st and (b) send a private EdStem post (not an email) requesting a regrade by midnight tonight 05/01/2025.

### ic14 
* ic14: I have not yet started on grading this one.
  * This one required you to make a post in your [Midterm Group Chat Channel](https://docs.google.com/spreadsheets/d/1oLPWP0j1jvx7eLq7kcQc15PiSDUwjlnZAolUVSHWBOQ/edit?gid=1128224438#gid=1128224438) following the instructions on [this Ed Post](https://edstem.org/us/courses/77825/discussion/6634588)
  * You'll get credit if I see a post in your chat channel.
  * If you haven't done it yet, do it before 8pm tonight 05/01/2025 to ensure full credit.
  
### ic15: Today's individual participation assignment

A simple one today, on Gradescope.  

* <https://www.gradescope.com/courses/1014383/assignments/6178129>
* [Five Minute Countdown](https://www.youtube.com/watch?v=_W0bSen8Qjg)

### ic16: Group participation assignment

See: <https://ucsb.instructure.com/courses/25658/assignments/360359>

* Try to get this done in class today (I'll give you the last 15 minutes to work on it).
* You have until midnight Friday in case you need to try to track down people that are not here.
* Grace period until midnight Saturday (no late penalty, but also no further extensions).

## Outline

* Review of `print` vs. `return`
* Some slides about group, pivot and join
* Some Time to work on ic16


## Setting up our course table

These first few cells are just setting up our course table with the usual columns so that we can use it as an example.

In [4]:
courseTable = Table.read_table("data/ucsb-s25-courses.csv")

In [5]:
def courseIdToDept(courseId):
    return courseId[0:8].strip()

def courseIdToCourseNum(courseId):
    return courseId[8:13].strip()

def courseIdToSuffix(courseId):
    return courseId[14:].strip()

def isLecture(sectionNum):
    # if section number is divisible by 100, it's a lecture
    return sectionNum % 100 == 0

In [6]:
courses = courseTable.with_columns(
    "dept", courseTable.apply(courseIdToDept, 0),
    "courseNum", courseTable.apply(courseIdToCourseNum, 0),
    "suffix", courseTable.apply(courseIdToSuffix, 0),
    "isLecture", courseTable.apply(isLecture, "SECTION")
)


In [8]:
csLectures = courses \
   .where("dept",are.equal_to("CMPSC")) \
   .where("isLecture",are.equal_to(True)) \
   .where("STATUS",are.not_equal_to("Closed"))

## Print vs. Return Review

Remember these two function definitions:

In [11]:
def addNumbers(a, b):
    return a + b

def sumNumbers(a, b):
    print(a + b)

Suppose we use call expressions with each of these:

In [15]:
# What happens when I run this cell?
result1 = addNumbers(3, 8)
result1 * 2

22

In [14]:
# What happens when I run this cell?
result2 = sumNumbers(2, 5)
result2 * 2

7


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

In [ ]:
Lets pause to understand why.

One way to see the difference is what happens when we assign the result.  Let's look at the values, and the types we get back:

## Review: summary of print vs. return

<table>
    <thead>
        <tr>
            <th>With Return</th>
            <th>With Print</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td markdown="1">
<pre>
def addNumbers(a, b):            
    return a + b
</pre>
            </td>
            <td markdown="1">
<pre>
def sumNumbers(a, b):         
    print(a + b)
</pre>
            </td>
        </tr>
        <tr>
            <td>return doesn't require ( ) </td>
            <td>print requires ( ) </td>
        </tr>
        <tr>
            <td>Assigning to variable <em>works</em>; the sum is <em>returned</em></td>
            <td>Assigning to a variable <em>doesn't work</em>; <code>None</code> is returned</td>
        </tr>
        <tr>
            <td>Result can be used in other expressions (e.g. `addNumbers(3,4)+1` gives 8</td>
            <td>Result CANNOT be used in other expressions (e.g. `sumNumbers(3,4)+1` gives an error</td>
        </tr>
    </tbody>
</table>

## *Every* function returns a value, but it's sometimes `None`



One way to understand this more easily is if you memorize this rule:
* If a function encounters a `return` statement, the expression after the `return` is what we *get* as the *result* of the function call, i.e. the value we can assign to a variable.
* If a function *has not return* statement, then at the end, there is an implied statement like this:
  ```
  return None
  ```


As a result, this is what happens:
<table>
    <thead>
        <tr>
            <th>If you write this function:</th>
            <th>What you get is actually this function:</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td markdown="1">
<pre>
def sumNumbers(a, b):         
    print(a + b)
</pre>
            </td>
            <td markdown="1">
<pre>
def sumNumbers(a, b):         
    print(a + b)
    return None
</pre>
            </td>
        </tr>
        <tr>
            <td>A function with no explicit <code>return</code> statement... </td>
            <td>... turns into one with <code>return None</code> as the last line </td>
        </tr>
    </tbody>
</table>


That doesn't mean that functions that `return None` are bad, or useless!  

It just means we have to be aware of this behavior.

    



BUT this doesn't work if you try it with functions that just *print* the sum:

# Grouping and Pivots

Slides: <https://docs.google.com/presentation/d/1FYg-mE8X0H5X0BHUUGlYIc0MAM9WzX8CtoVWGIrrg9k/edit?usp=sharing>

In [16]:
courses.labels

('COURSEID',
 'ENROLLED',
 'GES',
 'INSTRUCTOR',
 'MAXENROLL',
 'QUARTER',
 'SECTION',
 'STATUS',
 'dept',
 'courseNum',
 'suffix',
 'isLecture')

In [18]:
courses.num_rows

6879